# Ampliando el Transmetro

In [76]:
import pandas as pd
import numpy as np

In [77]:
capa = pd.read_csv("result.csv")
capa = capa[["MUNICIPIO", "LUGPOB", "NOMBRE_POB", "longitud", "latitud", "poblacion"]]
capa.head()

,MUNICIPIO,LUGPOB,NOMBRE_POB,longitud,latitud,poblacion
0,Guatemala,101001,Zona Municipal 1,-90.51200,14.637021,36236
1,Guatemala,101002,Zona Municipal 2,-90.51630,14.650479,19811
2,Guatemala,101003,Zona Municipal 3,-90.52742,14.628720,30995
3,Guatemala,101004,Zona Municipal 4,-90.51731,14.619216,1790
4,Guatemala,101005,Zona Municipal 5,-90.51020,14.618292,41441


In [78]:
municipios = ["Mixco"]
capa = capa.loc[capa["MUNICIPIO"].isin(municipios)]
capa.head()

,MUNICIPIO,LUGPOB,NOMBRE_POB,longitud,latitud,poblacion
485,Mixco,108001,Mixco,-90.60886,14.631747,15567
486,Mixco,108002,10 De Mayo,-90.60068,14.658637,699
487,Mixco,108003,14 De Octubre,-90.59187,14.627522,214
488,Mixco,108004,Primero De Julio,-90.56839,14.669567,13081
489,Mixco,108007,Alameda Del Encinal I,-90.60609,14.641889,357


In [79]:
# Algunos datos relevantes:
print(len(capa))
print(sum(capa["poblacion"]))

442
348407


# Geolocalización Rutas Principales
A continuación tenemos un código para hallar varios puntos de geolocalización para ubicar a grandes rasgos vias principales por donde podría pasar el transmetro. En particular para Mixco por ejemplo, tenemos el Bulevar de San Cristobal y la Roosevelt.

In [80]:
import requests

# Coordenadas de inicio y fin
inicio = [14.617513, -90.606171] 
final= [14.613316, -90.535345] 

# Se realiza el request según la documentación de OSMR 
url = f"http://router.project-osrm.org/route/v1/car/{inicio[1]},{inicio[0]};{final[1]},{final[0]}?alternatives=false&steps=true&geometries=polyline&overview=simplified&annotations=true"
response = requests.get(url)

# Capturamos el resultado del request.
if response.status_code == 200:
    geo = response.json()
else:
    print("Error occurred:", response.text)


Una vez que el request se halla realizado, en dado caso de que tal tarea halla sido exitosa, obtenemos un json, donde la llave "geometry" dentro de "route" posee un polyline codificada. Luego, la decodificamos utilizando la paquetería polyline, resultando en un listado de pares ordenados (lat, long) que representan puntos intermedios en el trayecto del punto inicial al punto final proporcionado.  

In [84]:
import polyline
polyline_codificada = geo["routes"][0]["geometry"]
trayecto = polyline.decode(polyline_codificada)


In [87]:
trayecto

,lat,lon
0,14.61752,-90.60615
1,14.61959,-90.60499
2,14.62272,-90.60452
3,14.62400,-90.60496
4,14.62534,-90.60686
5,14.62599,-90.60703
6,14.62647,-90.60659
7,14.62836,-90.60006
8,14.62884,-90.59930
9,14.62958,-90.59896


A modo de chequeo, escribimos estos datos en un archivo de excel y por medio de https://www.google.com/maps/d/ uno puede cargar este archivo y verificar los puntos sobre el polyline.

In [86]:
trayecto = [pd.DataFrame({"lat" : punto[0], "lon" : punto[1]}, index= [trayecto.index(punto)] ) for punto in trayecto]
trayecto = pd.concat(trayecto)
trayecto.to_excel("Trayecto3.xlsx")